# Place for experimenting the progressive design

In [1]:
import os,sys
import yaml
import inspect
import importlib

sys.path.append('..')

import model_discovery.utils as U
from model_discovery.configs.gam_config import GAMConfig, GAMConfig_14M
from model_discovery.model.gab_composer import GABTree,ROOT_UNIT_TEMPLATE,GABUnit
# from model_discovery.evolution import  BuildEvolution

ckpt_dir = os.environ['CKPT_DIR']
db_dir = U.pjoin(ckpt_dir, 'test_composer', 'db')
test_tree = GABTree('TestTree', db_dir)

prompts_dir='../model_discovery/agents/prompts/'
dir_prompt = U.pjoin(prompts_dir, 'prompts.yaml')
with open(dir_prompt, 'r') as file:
   gu_prompts = yaml.safe_load(file)
gab_py = U.read_file(U.pjoin(prompts_dir,'gab_template.py'))
gam_py = U.read_file(U.pjoin(prompts_dir,'gam_prompt.py'))
GU_TEMPLATE = U.read_file(U.pjoin(prompts_dir,'gu_template.py'))
GAB_UNIT=inspect.getsource(GABUnit)

c:\Users\ChengJunyan1\anaconda3\envs\modis\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\ChengJunyan1\.cache\huggingface\token
Login successful


In [4]:
test_tree.path
test_tree.get_source('TestTree')

'# TestTree.py\n\nimport torch\nimport torch.nn as nn\n\nfrom model_discovery.model.utils.modules import GABUnit # DO NOT CHANGE THIS IMPORT STATEMENT #\n\n\n# YOU CAN IMPORT MORE MODULES HERE #\n\n# YOU CAN DEFINE MORE CLASSES OR FUNCTIONS HERE #\n\n\nclass TestTree(GABUnit): \n    """Generalized Autoregressive Block\n        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all current intermediate variables}\n        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}\n        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable\n    """\n    def __init__(self, embed_dim: int, device=None, dtype=None,**kwargs): # YOU CAN ADD MORE ARGUMENTS, BUT YOU HAVE TO HAVE embed_dim, device, dtype AS THE ARGUTMENTS #\n        # argv: list of hyperparameters\n        factory_kwargs = {"device": device, "dtype": dtype} # remember to pass it to all nn layers\n        super().__init__(embed_di

In [2]:
import model_discovery.agents.prompts.prompts as P
importlib.reload(P)

gu_system_prompt=P.GU_DESIGNER_SYSTEM.format(GAB_BASE=P.GAB_BASE,GAM_PY=gam_py)
start_root_prompt=P.GU_DESIGN_SCRATCH_RAW.format(GAB_UNIT=GAB_UNIT,GU_TEMPLATE=GU_TEMPLATE,ROOT_UNIT_TEMPLATE=ROOT_UNIT_TEMPLATE)

print(start_root_prompt)



You are now designing an autoregressive model block. The auto-regressive model
is complex, so we will break it down into smaller parts. A block is represented
as multiple nested units which are called the Generalized Autoregressive Block
Unit (GABUnit). Each GABUnit accepts a sequence of embeddings X and a dictionary
of intermediate variables Z as input, and outputs a sequence of embeddings Y and
a dictionary of new or updated intermediate variables Z_. Z_ is optional, when
it is provided, it will be used to update Z by Z.update(Z_). 

A GABUnit is inherited from nn.Module as follows:

```python class GABUnit(nn.Module): 
    """ 
    Instead of directly giving the full implementation of a GAB block, the agent need to 
    design a series of nested GAB units and construct the full GAB block as a pipeline of these units.

    GAB is fractal, like GAB itself, each GAB unit accepts X and Z as input and returns Y and Z as output.
    """ 
    def __init__(self, embed_dim: int):
        su